In [215]:
import numpy as np
import networkx as nx
import aocd
from string import ascii_lowercase as keys
from string import ascii_uppercase as doors
from collections import defaultdict

def create(data: str) -> nx.Graph:
    d = np.array(list(map(list,data.split("\n"))))
    g = nx.Graph()
    for spos, src in np.ndenumerate(d):
        for p in [[1,0], [-1,0], [0, 1], [0, -1]]:
            dpos = tuple(np.array(spos)+p)
            if dpos[0] in range(d.shape[0]) and dpos[1] in range(d.shape[1]):
                dest = d[dpos]
                if src != "#" and dest != "#":
                    g.add_edge(spos, dpos, weight=1)
                    if src in keys:    g.nodes[spos]["key"] = src
                    elif src in doors: g.nodes[spos]["door"] = src.lower()
                    g.nodes[spos]["typ"] = src
    
    # trim graph
    for node in g.copy().nodes:
        neigh = list(g.neighbors(node))
        if len(neigh)==2 and g.nodes[node]["typ"] == ".":
            weight = sum([g.edges[(node, n)]["weight"] for n in neigh])
            g.remove_node(node)
            g.add_edge(*neigh, weight=weight)
    return g

def bfs(data):
    # init data
    g = create(data)
    startpos = [pos for pos, typ in nx.get_node_attributes(g, "typ").items() if typ=="@"][0]
    jobs = [((0, 0, startpos, frozenset()))]
    pos_door = nx.get_node_attributes(g, "door")
    pos_key =  nx.get_node_attributes(g, "key")
    
    # build dict (src) -> (dest, doors_in_way, keys_in_way)
    key_dist = defaultdict(list)
    for p1, k1 in list(pos_key.items())+[[startpos, "@"]]:
        for p2, k2 in pos_key.items():
            if k1 != k2:
                sp = nx.shortest_path(g, p1, p2, weight="weight")
                blocked_by = set(sp) & (set(pos_door))
                blocked_by = set([pos_door[bb] for bb in blocked_by])
                
                blocking = set(sp) & set(pos_key) - set([p1,p2])
                blocking = set([pos_key[bb] for bb in blocking])
                
                dist = nx.shortest_path_length(g, p1, p2, weight="weight")
                key_dist[k1].append([k2, dist, blocked_by, blocking])
    
    # assumption: we can't "walk around doors"
    # and the shortest_path(shortest_path(keys)) is shorter 
    jobs = {(0, "@", frozenset())}
    for _ in range(len(pos_key)):  # iterate bfs for number of keys, as we need that many steps
        jobs = list(set(  # unique
            [(steps+dist, target, keys_hold|set([target]))  # add new dist-target-visited tuples
                for steps, start, keys_hold in jobs         # for all current start-tuples
                    for target, dist, blocked_by, blocking in key_dist[start]  # for all target-tuples
                        if target not in keys_hold          # if target is not already reached
                        and not blocked_by - keys_hold      # not blocked by a door without a key
                        and not blocking - keys_hold]))     # and not blocked by a key on the 
    return min([j[0] for j in jobs])
    
assert bfs("""#########
#b.A.@.a#
#########""")

assert bfs("""########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################""") == 81

In [204]:
aocd.submit(bfs(aocd.get_data(day=18)), day=18)

answer a: 5402
submitting for part b (part a is already completed)


aocd will not submit that answer again. You've previously guessed 5402 and the server responded:
That's not the right answer; your answer is too high.  If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit.  Please wait one minute before trying again. (You guessed 5402.) [Return to Day 18]
